##### THIS FILE IS USED TO GENERATE CUSTOMER-CENTERED DATA #####

In [7]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import math
import os
from dateutil.relativedelta import relativedelta
from sklearn.cluster import KMeans

In [8]:
# This is used to block warning messages
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
# Or ignore specific warnings by category (e.g., FutureWarnings)
warnings.filterwarnings("ignore", category=FutureWarning)
# This is used to edit image
from IPython.display import Image
from IPython.core.display import HTML 

In [9]:
# df = pd.read_excel(r'C:\Users\Admin\RBAC\Data.xlsx')
# df.to_csv('data_csv.csv', index=False)  # Replace 'your_csv_file.csv' with the desired CSV file path

In [10]:
df = pd.read_csv(r'C:\Users\Admin\RBAC\OriDataNew.csv').drop(columns='Unnamed: 0')

In [11]:
df

,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,CustomerGender,VoucherStatus,Province
0,0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,1753863,Unknown,No,Ho Chi Minh City
1,1,Take Away,STORE,2021-10-01 00:00:00+00:00,301782.0,1124050,Unknown,No,Hanoi
2,2,Take Away,WEBSITE,2021-10-01 00:00:00+00:00,319792.0,1626827,Male,No,Hanoi
3,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi
4,4,Delivery,STORE,2021-10-01 00:00:00+00:00,280031.0,2117237,Unknown,No,Hanoi
...,...,...,...,...,...,...,...,...,...
1397504,1397504,Delivery,WEBSITE,2023-07-01 00:00:00+00:00,87103.0,1401593,Unknown,No,Ho Chi Minh City
1397505,1397505,Take Away,STORE,2023-07-01 00:00:00+00:00,278920.0,1322074,Male,No,Hanoi
1397506,1397506,Take Away,WEBSITE,2023-07-01 00:00:00+00:00,364872.0,2038523,Unknown,No,Nothern Provinces
1397507,1397507,Delivery,APP,2023-07-01 00:00:00+00:00,1349201.0,1443231,Male,No,Southern Provinces


In [12]:
print(
    min(df['TransactionDate']),
    max(df['TransactionDate'])
)

2021-10-01 00:00:00+00:00 2023-07-01 00:00:00+00:00


In [14]:
len(df['CustomerID'].unique())

718169

In [15]:
customer = pd.DataFrame(df['CustomerID'].unique(),columns=['CustomerID'])

In [16]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['days_between_purchase'] = df.groupby('CustomerID')['TransactionDate'].diff().dt.days

In [17]:
df['days_between_purchase'] = df['days_between_purchase'].fillna(0)

In [18]:
days_between = df.groupby('CustomerID',as_index=False).agg({'days_between_purchase':['mean','max']})

In [19]:
days_between

CustomerID days_between_purchase       
                                   mean    max
0               0              0.000000    0.0
1               1              0.000000    0.0
2               8            120.000000  240.0
3              14              0.000000    0.0
4              16              0.000000    0.0
...           ...                   ...    ...
718164    2174066              0.000000    0.0
718165    2174072             27.333333   52.0
718166    2174073              0.000000    0.0
718167    2174075              0.000000    0.0
718168    2174083              0.000000    0.0

[718169 rows x 3 columns]

In [20]:
days_between.columns = ['CustomerID','mean_days_between_purchase','max_days_between_purchase']
days_between

,CustomerID,mean_days_between_purchase,max_days_between_purchase
0,0,0.000000,0.0
1,1,0.000000,0.0
2,8,120.000000,240.0
3,14,0.000000,0.0
4,16,0.000000,0.0
...,...,...,...
718164,2174066,0.000000,0.0
718165,2174072,27.333333,52.0
718166,2174073,0.000000,0.0
718167,2174075,0.000000,0.0


In [21]:
value = df.groupby('CustomerID',as_index=False).agg({'SalesAmount':'sum','BillID':'count'})
value['value_per_order'] = value['SalesAmount'] / value['BillID']
value.columns = ['CustomerID','TotalValue','CountOrder','ValuePerOrder']
value

,CustomerID,TotalValue,CountOrder,ValuePerOrder
0,0,411663.0,1,411663.000000
1,1,105503.0,1,105503.000000
2,8,1941696.0,2,970848.000000
3,14,267607.0,1,267607.000000
4,16,286558.0,1,286558.000000
...,...,...,...,...
718164,2174066,143685.0,1,143685.000000
718165,2174072,953618.0,3,317872.666667
718166,2174073,514503.0,1,514503.000000
718167,2174075,262489.0,1,262489.000000


In [22]:
df['Channel'].unique()

array(['Take Away', 'Delivery', 'Dine In'], dtype=object)

In [23]:
Channel_Delivery = df[df['Channel']=='Delivery'].groupby('CustomerID',as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'Channel_Delivery'})
Channel_TakeAway = df[df['Channel']=='Take Away'].groupby('CustomerID',as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'Channel_TakeAway'})
Channel_DineIn = df[df['Channel']=='Dine In'].groupby('CustomerID',as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'Channel_DineIn'})

In [24]:
Channel_Delivery

,CustomerID,Channel_Delivery
0,0,1
1,1,1
2,8,2
3,14,1
4,16,1
...,...,...
342543,2174051,1
342544,2174059,1
342545,2174072,1
342546,2174073,1


In [25]:
df['OrderFrom'].unique()

array(['CALL CENTER', 'STORE', 'WEBSITE', 'APP'], dtype=object)

In [26]:
OrderFrom_Store = df[df['OrderFrom']=='STORE'].groupby('CustomerID',as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_Store'})
OrderFrom_Web = df[df['OrderFrom']=='WEBSITE'].groupby('CustomerID',as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_Web'})
OrderFrom_CallCenter = df[df['OrderFrom']=='CALL CENTER'].groupby('CustomerID',as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_CallCenter'})
OrderFrom_App = df[df['OrderFrom']=='APP'].groupby('CustomerID',as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_App'})

In [27]:
print(
    len(OrderFrom_Store),
    len(OrderFrom_Web),
    len(OrderFrom_CallCenter),
    len(OrderFrom_App),
)

424014 196099 121214 92004


In [28]:
demographic = df[['CustomerID','CustomerGender','Province']].drop_duplicates(subset=['CustomerID'])

In [29]:
lifetime = df.groupby('CustomerID', as_index=False).agg({'TransactionDate':['min','max']})
lifetime.columns = ['CustomerID','First_Order_Day','Last_Order_Day']
lifetime['LifeTime'] = (lifetime['Last_Order_Day'] -  lifetime['First_Order_Day']).dt.days
lifetime

,CustomerID,First_Order_Day,Last_Order_Day,LifeTime
0,0,2021-12-24 00:00:00+00:00,2021-12-24 00:00:00+00:00,0
1,1,2022-03-07 00:00:00+00:00,2022-03-07 00:00:00+00:00,0
2,8,2022-07-11 00:00:00+00:00,2023-03-08 00:00:00+00:00,240
3,14,2023-05-27 00:00:00+00:00,2023-05-27 00:00:00+00:00,0
4,16,2022-09-16 00:00:00+00:00,2022-09-16 00:00:00+00:00,0
...,...,...,...,...
718164,2174066,2023-01-03 00:00:00+00:00,2023-01-03 00:00:00+00:00,0
718165,2174072,2022-09-13 00:00:00+00:00,2022-12-04 00:00:00+00:00,82
718166,2174073,2023-02-12 00:00:00+00:00,2023-02-12 00:00:00+00:00,0
718167,2174075,2022-12-10 00:00:00+00:00,2022-12-10 00:00:00+00:00,0


In [30]:
# MERGING ALL TOGETHER
final_data = customer.merge(days_between, 
               on='CustomerID',how='left').merge(value,
                                      on='CustomerID',how='left').merge(Channel_Delivery,
                                                             on='CustomerID',how='left').merge(Channel_TakeAway,
                                                                     on='CustomerID',how='left').merge(Channel_DineIn,
                                                                             on='CustomerID',how='left').merge(OrderFrom_Store,
                                                                                     on='CustomerID',how='left').merge(OrderFrom_Web,
                                                                                             on='CustomerID',how='left').merge(OrderFrom_CallCenter,
                                                                                                     on='CustomerID',how='left').merge(OrderFrom_App,
                                                                                                             on='CustomerID',how='left').merge(demographic,
                                                                                                                        on='CustomerID',how='left').merge(lifetime,
                                                                                                                                on='CustomerID',how='left')

In [31]:
final_data = final_data.fillna(0)
final_data.to_csv('Customer_data.csv')